### Perform envelope cross correlation between the two OOI stations
- In parallel over days using dask
- NOTE that the channel used is hard coded in the "calc_cc_envelope" function - need to change it and rerun for different channels!
- Follows approach of Wech et al. 2013

In [1]:
import os
import pickle

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import numpy as np
import obspy
from obspy.clients.fdsn.client import Client 
import sklearn
from sklearn.decomposition import FastICA

import obspy

import pandas as pd

import obspy.signal.cross_correlation
client = Client('IRIS')
import dask
from dask.diagnostics import ProgressBar

In [40]:
def calc_cc_envelope(t1,t2):
    """
    t1 - start time of the time window to process, obspy UTCDateTime object
    t2 - end time, obspy UTCDateTime object
    
    Outputs:
    List of the cross correlation values, lag times, and start times of each cross-correlated window
    """
    
    
    try:
        ### CALCULATING ENVELOPES ###
        st = client.get_waveforms("OO",'HYSB1', "*", "HHN", t1-5, t2+5);
        st.resample(200).merge(fill_value='interpolate')
        data = st[0].data
        npts = st[0].stats.npts
        samprate = st[0].stats.sampling_rate

        # Skip if there is not enough data
        if len(data) < (86400*0.99*samprate):
            return None,None,None

        # Filtering the Stream object
        st_filt = st.copy()
        st_filt.taper(0.05,max_length=5)
        st_filt.filter('bandpass', freqmin=3, freqmax=10)
        # Envelope of filtered data
        data_envelope = obspy.signal.filter.envelope(st_filt[0].data)
        # Low-pass filter of 5 s
        lowpass = obspy.signal.filter.lowpass(data_envelope,1/5,samprate)
        # Decimate to 1 Hz
        # New sampling rate is old sampling rate divided by decimation_factor.
        decimation_factor = int(samprate)
        envelope1 = obspy.signal.filter.integer_decimation(lowpass,decimation_factor)
        # Trim!
        envelope1 = envelope1[5:-5]

        st = client.get_waveforms("OO",'HYS14', "*", "HHN", t1-5, t2+5);
        st.resample(200).merge(fill_value='interpolate')
        data = st[0].data
        npts = st[0].stats.npts
        samprate = st[0].stats.sampling_rate

        # Filtering the Stream object
        st_filt = st.copy()
        st_filt.taper(0.05,max_length=5)
        st_filt.filter('bandpass', freqmin=3, freqmax=10)
        # Envelope of filtered data
        data_envelope = obspy.signal.filter.envelope(st_filt[0].data)
        # Low-pass filter of 5 s
        lowpass = obspy.signal.filter.lowpass(data_envelope,1/5,samprate)
        # Decimate to 1 Hz
        # New sampling rate is old sampling rate divided by decimation_factor.
        decimation_factor = int(samprate)
        envelope2 = obspy.signal.filter.integer_decimation(lowpass,decimation_factor)
        # Trim!
        envelope2 = envelope2[5:-5]

        ### CROSS-CORRELATING ENVELOPES ###
        spacing = 300
        overlap = 0.5
        data_length = min([len(envelope1)-1,len(envelope2)-1])
        steps = np.arange(0,data_length,int(spacing*overlap))


        shifts = []
        values = []
        times = []
        for start in steps:

            stop = start + spacing

            cc = obspy.signal.cross_correlation.correlate(envelope1[start:stop],envelope2[start:stop],shift=int(10*samprate))
            shift, value = obspy.signal.cross_correlation.xcorr_max(cc)


            values.append(value)
            shifts.append(shift)
            times.append(st[0].stats.starttime+start)
    except:
        
        return None,None,None
    
    return values,shifts,times



### Set up to run in parallel over days

In [41]:
t1 = obspy.UTCDateTime("2018-07-01T00:00:00.000")
t2 = obspy.UTCDateTime("2019-07-01T00:00:00.000")
data_time_length = 24 * 60 * 60
time_bins = pd.date_range(start=t1.datetime, end=t2.datetime, freq='d')

In [42]:
@dask.delayed
def loop_days(t1,t2):
    return calc_cc_envelope(obspy.UTCDateTime(t1),obspy.UTCDateTime(t2),'HYSB1','HYS14')

In [43]:
lazy_results = [loop_days(time_bins[i],time_bins[i+1]) for i,t in enumerate(time_bins[:-1])]

In [44]:
%%time
with ProgressBar():
    results = dask.compute(lazy_results)

[########################################] | 100% Completed |  1hr 22min 29.1s
CPU times: user 6h 45min 17s, sys: 12min 19s, total: 6h 57min 36s
Wall time: 1h 22min 29s


### Parse results and write to file

In [45]:
# Parse results
values = []
shifts = []
times = []
for r in results[0]:
    if r[0]!=None:
        values.extend(r[0])
        shifts.extend(r[1])
        times.extend(r[2])
# Sort results
sort_ind = np.argsort(times)
times = [times[i] for i in sort_ind]
values = [values[i] for i in sort_ind]
shifts = [shifts[i] for i in sort_ind]

In [46]:
file_name = 'HYSB1_HYS14_HHN_cc_10s_shift.pickle'
with open(file_name, 'wb') as handle:
    pickle.dump([times,values,shifts],handle)   